In [2]:
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils import shuffle
from scipy.stats import uniform
from scipy.stats import randint
import cv2

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from extract_data import extract_data

np.random.seed(52384)

In [3]:
def pca_reduce(x_train, x_test, num_components):
  # Standardize the data
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(x_train)
  X_test_scaled = scaler.transform(x_test)

  # Perform PCA
  pca = PCA(n_components=num_components)
  X_train_pca = pca.fit_transform(X_train_scaled)
  X_test_pca = pca.transform(X_test_scaled)
  return X_train_pca, X_test_pca

In [4]:
# model = SVC()
# model.fit(x_train_gs, y_train)
# # training
# y_pred = model.predict(x_train_gs)
# print(accuracy_score(y_train, y_pred))
# print(classification_report(y_train, y_pred))
# # testing
# y_pred = model.predict(x_test_gs)
# print(accuracy_score(y_test, y_pred))
# print(classification_report(y_test, y_pred))

# model = SVC()
# model.fit(X_train_pca, y_train)
# y_pred = model.predict(X_train_pca)
# print(accuracy_score(y_train, y_pred))
# print(classification_report(y_train, y_pred))
# y_pred = model.predict(X_test_pca)
# print(accuracy_score(y_test, y_pred))
# print(classification_report(y_test, y_pred))

## 1. One subject

In [5]:
# extract train and test data
x_train, y_train, x_test, y_test = extract_data('../ds000105_R2.0.2', sub_num=1)
x_train2 = x_train.reshape(40*64*x_train.shape[2], x_train.shape[3])
x_test2 = x_test.reshape(40*64*x_train.shape[2], x_test.shape[3])
x_train2 = x_train2.T
x_test2 = x_test2.T
mean_train = np.mean(x_train2, axis=0)
std_train = np.std(x_train2, axis=0)
x_train_norm = (x_train2 - mean_train) / std_train
x_test_norm = (x_test2 - mean_train) / std_train

In [6]:
# denoise: Gaussian smoothing
x_train_gs = cv2.GaussianBlur(x_train_norm, (5, 5), 0)
x_test_gs = cv2.GaussianBlur(x_test_norm, (5, 5), 0)
# PCA reduce order
x_train_pca, x_test_pca=pca_reduce(x_train_gs,x_test_gs,150)

In [7]:
svm = SVC(random_state=42)
hyper_param_svm = {
    'C': np.logspace(-3, 5, 9),
    'kernel': ['poly','rbf'],
    'degree': [1, 2, 3, 4],
    'gamma': ['scale', 'auto'] + list(np.logspace(-8, 3, 12)),
}
model_cv_svm = RandomizedSearchCV(svm,
                        param_distributions = hyper_param_svm,
                        n_iter = 100,
                        scoring = 'accuracy',
                        cv = 10, 
                        n_jobs = -1)

In [8]:
rf = RandomForestClassifier(random_state=42)
hyper_param_rf = {
    'n_estimators': randint(50, 500),
    'max_depth': [None] + list(range(5, 30, 5)),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'max_features': ['sqrt', 'log2'] + [x_train_gs.shape[1]],
    'criterion': ['gini', 'entropy'],
    'bootstrap': [True, False]
}
model_cv_rf = RandomizedSearchCV(rf, 
                        param_distributions = hyper_param_rf,
                        n_iter = 100,
                        scoring = 'accuracy',
                        cv = 10, 
                        n_jobs = -1)

In [9]:
knn = KNeighborsClassifier()
hyper_param_knn = {'n_neighbors': range(1,150,10),
                   'weights': ['uniform', 'distance']}
model_cv_knn = GridSearchCV(estimator=knn,
                        param_grid = hyper_param_knn,
                        scoring = 'accuracy', 
                        cv = 10, 
                        n_jobs = -1)

### 1.1 Only smoothing

#### 1.1.1 SVM

In [43]:
search1 = model_cv_svm.fit(x_train_gs, y_train)
search1.best_params_

{'kernel': 'rbf', 'gamma': 1e-06, 'degree': 4, 'C': 100000.0}

In [44]:
print('The best average accuracy over CV is:')
print(search1.cv_results_['mean_test_score'][search1.cv_results_['rank_test_score']==1][0])

The best average accuracy over CV is:
0.6431578947368422


In [45]:
model = SVC(kernel=search1.best_params_.get('kernel'), gamma=search1.best_params_.get('gamma'), C=search1.best_params_.get('C'), degree=search1.best_params_.get('degree'))
model.fit(x_train_gs, y_train)

SVC(C=100000.0, degree=4, gamma=1e-06)

In [46]:
y_pred = model.predict(x_test_gs)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8235294117647058
              precision    recall  f1-score   support

         0.0       1.00      0.57      0.73         7
         1.0       0.77      1.00      0.87        10

    accuracy                           0.82        17
   macro avg       0.88      0.79      0.80        17
weighted avg       0.86      0.82      0.81        17



#### 1.1.2 Random Forest

In [ ]:
search2 = model_cv_rf.fit(x_train_gs, y_train)
search2.best_params_

In [ ]:
print('The best average accuracy over CV is:')
print(search2.cv_results_['mean_test_score'][search2.cv_results_['rank_test_score']==1][0])

In [ ]:
model = RandomForestClassifier(n_estimators=search2.best_params_.get('n_estimators'), 
                               max_depth=search2.best_params_.get('max_depth'), 
                               min_samples_split=search2.best_params_.get('min_samples_split'), 
                               min_samples_leaf=search2.best_params_.get('min_samples_leaf'),
                               max_features=search2.best_params_.get('max_features'),
                               criterion=search2.best_params_.get('criterion'),
                               bootstrap=search2.best_params_.get('bootstrap'))
model.fit(x_train_gs, y_train)

In [ ]:
y_pred = model.predict(x_test_gs)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

#### 1.1.3 KNN

In [53]:
search3 = model_cv_knn.fit(x_train_gs, y_train)
search3.best_params_

{'n_neighbors': 101, 'weights': 'distance'}

In [54]:
print('The best average accuracy over CV is:')
print(search3.cv_results_['mean_test_score'][search3.cv_results_['rank_test_score']==1][0])

The best average accuracy over CV is:
0.5623684210526316


In [55]:
model = KNeighborsClassifier(n_neighbors=search3.best_params_.get('n_neighbors'),
                           weights=search3.best_params_.get('weights'))
model.fit(x_train_gs, y_train)

KNeighborsClassifier(n_neighbors=101, weights='distance')

In [56]:
y_pred = model.predict(x_test_gs)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8235294117647058
              precision    recall  f1-score   support

         0.0       0.70      1.00      0.82         7
         1.0       1.00      0.70      0.82        10

    accuracy                           0.82        17
   macro avg       0.85      0.85      0.82        17
weighted avg       0.88      0.82      0.82        17



### 1.2 First smoothing, and then PCA

#### 1.2.1 SVM

In [58]:
search4 = model_cv_svm.fit(x_train_pca, y_train)
search4.best_params_

{'kernel': 'poly', 'gamma': 1.0, 'degree': 1, 'C': 0.01}

In [59]:
print('The best average accuracy over CV is:')
print(search4.cv_results_['mean_test_score'][search4.cv_results_['rank_test_score']==1][0])

The best average accuracy over CV is:
0.6534210526315789


In [60]:
model = SVC(kernel=search4.best_params_.get('kernel'), gamma=search4.best_params_.get('gamma'), C=search4.best_params_.get('C'), degree=search4.best_params_.get('degree'))
model.fit(x_train_pca, y_train)

SVC(C=0.01, degree=1, gamma=1.0, kernel='poly')

In [61]:
y_pred = model.predict(x_test_pca)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8235294117647058
              precision    recall  f1-score   support

         0.0       1.00      0.57      0.73         7
         1.0       0.77      1.00      0.87        10

    accuracy                           0.82        17
   macro avg       0.88      0.79      0.80        17
weighted avg       0.86      0.82      0.81        17



#### 1.2.2 Random Forest

In [ ]:
search5 = model_cv_rf.fit(x_train_pca, y_train)
search5.best_params_

In [ ]:
print('The best average accuracy over CV is:')
print(search5.cv_results_['mean_test_score'][search5.cv_results_['rank_test_score']==1][0])

In [ ]:
model = RandomForestClassifier(n_estimators=search5.best_params_.get('n_estimators'), 
                               max_depth=search5.best_params_.get('max_depth'), 
                               min_samples_split=search5.best_params_.get('min_samples_split'), 
                               min_samples_leaf=search5.best_params_.get('min_samples_leaf'),
                               max_features=search5.best_params_.get('max_features'),
                               criterion=search5.best_params_.get('criterion'),
                               bootstrap=search5.best_params_.get('bootstrap'))
model.fit(x_train_pca, y_train)

In [ ]:
y_pred = model.predict(x_test_pca)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

#### 1.2.3 KNN

In [74]:
search6 = model_cv_knn.fit(x_train_pca, y_train)
search6.best_params_

{'n_neighbors': 1, 'weights': 'uniform'}

In [75]:
print('The best average accuracy over CV is:')
print(search6.cv_results_['mean_test_score'][search6.cv_results_['rank_test_score']==1][0])

The best average accuracy over CV is:
0.573421052631579


In [76]:
model = KNeighborsClassifier(n_neighbors=search6.best_params_.get('n_neighbors'),
                           weights=search6.best_params_.get('weights'))
model.fit(x_train_pca, y_train)

KNeighborsClassifier(n_neighbors=1)

In [77]:
y_pred = model.predict(x_test_pca)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.7647058823529411
              precision    recall  f1-score   support

         0.0       0.71      0.71      0.71         7
         1.0       0.80      0.80      0.80        10

    accuracy                           0.76        17
   macro avg       0.76      0.76      0.76        17
weighted avg       0.76      0.76      0.76        17



### 1.3 No smoothing and PCA

#### 1.3.1 SVM

In [62]:
search7 = model_cv_svm.fit(x_train_norm, y_train)
search7.best_params_

{'kernel': 'rbf', 'gamma': 1e-05, 'degree': 1, 'C': 10000.0}

In [63]:
print('The best average accuracy over CV is:')
print(search7.cv_results_['mean_test_score'][search7.cv_results_['rank_test_score']==1][0])

The best average accuracy over CV is:
0.9692105263157895


In [64]:
model = SVC(kernel=search7.best_params_.get('kernel'), gamma=search7.best_params_.get('gamma'), C=search7.best_params_.get('C'), degree=search7.best_params_.get('degree'))
model.fit(x_train_norm, y_train)

SVC(C=10000.0, degree=1, gamma=1e-05)

In [65]:
y_pred = model.predict(x_test_norm)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

1.0
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         7
         1.0       1.00      1.00      1.00        10

    accuracy                           1.00        17
   macro avg       1.00      1.00      1.00        17
weighted avg       1.00      1.00      1.00        17



#### 1.3.2 Random Forest

In [ ]:
search8 = model_cv_rf.fit(x_train_norm, y_train)
search8.best_params_

In [ ]:
print('The best average accuracy over CV is:')
print(search8.cv_results_['mean_test_score'][search8.cv_results_['rank_test_score']==1][0])

In [ ]:
model = RandomForestClassifier(n_estimators=search8.best_params_.get('n_estimators'), 
                               max_depth=search8.best_params_.get('max_depth'), 
                               min_samples_split=search8.best_params_.get('min_samples_split'), 
                               min_samples_leaf=search8.best_params_.get('min_samples_leaf'),
                               max_features=search8.best_params_.get('max_features'),
                               criterion=search8.best_params_.get('criterion'),
                               bootstrap=search8.best_params_.get('bootstrap'))
model.fit(x_train_norm, y_train)

In [ ]:
y_pred = model.predict(x_test_norm)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

#### 1.3.3 KNN

In [70]:
search9 = model_cv_knn.fit(x_train_norm, y_train)
search9.best_params_

{'n_neighbors': 1, 'weights': 'uniform'}

In [71]:
print('The best average accuracy over CV is:')
print(search9.cv_results_['mean_test_score'][search9.cv_results_['rank_test_score']==1][0])

The best average accuracy over CV is:
0.9444736842105262


In [72]:
model = KNeighborsClassifier(n_neighbors=search9.best_params_.get('n_neighbors'),
                           weights=search9.best_params_.get('weights'))
model.fit(x_train_norm, y_train)

KNeighborsClassifier(n_neighbors=1)

In [73]:
y_pred = model.predict(x_test_norm)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

1.0
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         7
         1.0       1.00      1.00      1.00        10

    accuracy                           1.00        17
   macro avg       1.00      1.00      1.00        17
weighted avg       1.00      1.00      1.00        17

